#### Assign BA counts by County

The current version of assigning BA counts to a bus uses state-level population fractions, for example, if a BA is known to offer service in both WA and OR, e.g., BA=BPAT, then the BA counts, for example, are

BPATcount(WA) = BPATtotal * pop(WA)/(pop(WA) + pop(OR)
demandCount(WA) = sum(all fractional BA counts assigned to WA)

However, we actually know what counties each BA serves, so instead of using state-level population, we can use county level population. So 

* Demand in LoadZone = sumOverCountiesInLoadZone(demandPerCounty)
* demandPerCounty = sumOverBAs(BAcount(County))
* BAcounty(County) = BAtotal * CountyPopulation/sumOverCountyPopulationServedByBA

So we need to make a list of Counties per LoadZone and the corresponing populations for those Counties.
                   
<img src="BA-county_array.png">

Note that the array values have to be updated as county census data are updated anda as BA-county assignments change.

In [1]:
import pandas as pd
import numpy as np

In [2]:
WA = ['AVA', 'BPAT','CHPD','DOPD','GCPD','PSEI','SCL','TPWR','AVA', 'BPAT']
OR = ['BPAT','GRID','IPCO','PACW','PGE']
CAnorth = ['BANC','CISO','PACW','BPAT']
CABayArea = ['CISO']
CAcentral = ['CISO', 'TIDC']
CAsw = ['CISO', 'LDWP']
CAse = ['CISO', 'IID']
NV = ['NEVP']
AZ = ['AZPS','DEAA','GRIF','GRMA','HGMA','PNM','SRP','TEPC','WALC']
UT = ['PACE']
NM = ['EPE','PNM']
CO = ['PSCO','WACM']
WY = ['PACE','WACM']
ID = ['AVA','BPAT','IPCO','PACE']
MT = ['BPAT','GWA','NWMT','WAUW','WWA'] 
ElPaso = ['EPE']

ba_all = sorted(list(set().union(WA,OR,CAnorth,CABayArea,CAcentral,CAsw,CAse,NV,AZ,UT,NM,CO,WY,ID,MT,ElPaso)))

In [3]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

In [4]:
dir_861 = 'U:\\src\\PreREISE\\prereise\\gather\\demanddata\\EIA\\demo\\data\\f8612016\\'
dir_pop = 'U:\\src\\PreREISE\\prereise\\gather\\demanddata\\EIA\\demo\\data\\US_Counties_Population\\'
dir_BA =  'U:\\src\\PreREISE\\prereise\\gather\\demanddata\\EIA\\test\\data\\'

service_territory_file = 'Service_Territory_2016.xlsx'  #get utility-county mapping
ba_counts_file = 'Sales_Ult_Cust_2016.xlsx'  #Get utility-BAname mapping
ba_id_map_file = 'Balancing_Authority_2016.xlsx'  #use for checking final join: which BAs by state

#This assumes data has been through outlier fixing for now
ba_demand_file = 'BA_2016.csv'

#This file is edited from original due to mysterious read problems
us_counties_pop_file = 'USCountyPop.csv'

service_territory =  pd.read_excel(io = dir_861 + '/' + service_territory_file, 
                                   header = 0,
                                   dtype = {'Utility Number': str, 'County': str, 'State': str}
                                  )
ba_counts = pd.read_excel(io = dir_861 + '/' + ba_counts_file, 
                          header = 0, usecols = 'B,G,I,W', skiprows = [0,1], 
                          dtype = {'Utility Number': str, 'BA_CODE': str, 'State': str}
                          #sheet_name = ['States']
                         )
counties_pop = pd.read_csv(dir_pop + '/' + us_counties_pop_file, header=0)
ba_id_map = pd.read_excel(io = dir_861 + '/' + ba_id_map_file, header = 0)
ba_demand = pd.read_csv(dir_BA + '/' + ba_demand_file, header = 0, parse_dates = True, index_col = 'UTC Time')

In [5]:
service_territory.head()

,Data Year,Utility Number,Utility Name,State,County
0,2016,34,City of Abbeville - (SC),SC,Abbeville
1,2016,55,City of Aberdeen - (MS),MS,Monroe
2,2016,59,City of Abbeville - (LA),LA,Vermilion
3,2016,84,A & N Electric Coop,MD,Somerset
4,2016,84,A & N Electric Coop,VA,Accomack


In [6]:
ba_counts.head()

,Utility Number,State,BA_CODE,Megawatthours
0,55,AL,TVA,0
1,55,MS,TVA,201106
2,59,LA,MISO,132078
3,84,MD,PJM,2565
4,84,VA,PJM,672045


In [7]:
ba_county = pd.merge(service_territory, ba_counts, how='outer', on ='Utility Number')

In [8]:
ba_county.head(10)

,Data Year,Utility Number,Utility Name,State_x,County,State_y,BA_CODE,Megawatthours
0,2016.0,34,City of Abbeville - (SC),SC,Abbeville,NaN,NaN,NaN
1,2016.0,55,City of Aberdeen - (MS),MS,Monroe,AL,TVA,0
2,2016.0,55,City of Aberdeen - (MS),MS,Monroe,MS,TVA,201106
3,2016.0,59,City of Abbeville - (LA),LA,Vermilion,LA,MISO,132078
4,2016.0,84,A & N Electric Coop,MD,Somerset,MD,PJM,2565
5,2016.0,84,A & N Electric Coop,MD,Somerset,VA,PJM,672045
6,2016.0,84,A & N Electric Coop,VA,Accomack,MD,PJM,2565
7,2016.0,84,A & N Electric Coop,VA,Accomack,VA,PJM,672045
8,2016.0,84,A & N Electric Coop,VA,Northampton,MD,PJM,2565
9,2016.0,84,A & N Electric Coop,VA,Northampton,VA,PJM,672045


In [9]:
ba_county.loc[ba_county['BA_CODE'].isin(ba_all)].sort_values('BA_CODE').head(20)

,Data Year,Utility Number,Utility Name,State_x,County,State_y,BA_CODE,Megawatthours
14165,2016.0,20169,Avista Corp,MT,Sanders,ID,AVA,2930223
14161,2016.0,20169,Avista Corp,ID,Nez Perce,WA,AVA,5578322
14162,2016.0,20169,Avista Corp,ID,Shoshone,ID,AVA,2930223
20943,NaN,99999,NaN,NaN,NaN,ID,AVA,78753
14163,2016.0,20169,Avista Corp,ID,Shoshone,MT,AVA,785
14164,2016.0,20169,Avista Corp,ID,Shoshone,WA,AVA,5578322
5792,2016.0,10454,Kootenai Electric Cooperative,ID,Kootenai,ID,AVA,419675
14166,2016.0,20169,Avista Corp,MT,Sanders,MT,AVA,785
14167,2016.0,20169,Avista Corp,MT,Sanders,WA,AVA,5578322
14168,2016.0,20169,Avista Corp,WA,Adams,ID,AVA,2930223


In [10]:
counties_pop.drop(columns = ['GEO.id', 'GEO.id2', 'rescen42010', 'resbase42010',
       'respop72010', 'respop72011', 'respop72012', 'respop72013',
       'respop72014', 'respop72015','respop72017'], inplace = True)

In [11]:
counties_pop.columns

Index(['GEO.display-label', 'respop72016'], dtype='object')

In [12]:
counties_pop.head()

,GEO.display-label,respop72016
0,"Autauga County, Alabama",55278
1,"Baldwin County, Alabama",207509
2,"Barbour County, Alabama",25774
3,"Bibb County, Alabama",22633
4,"Blount County, Alabama",57562


In [13]:
counties_pop['County'] = counties_pop['GEO.display-label'].str.split(',',expand=True)[0].str.replace(' County','')
counties_pop['State'] = counties_pop['GEO.display-label'].str.split(',',expand=True)[1].str.replace('^ ','')
counties_pop['State'] = counties_pop['State'].apply(lambda x: us_state_abbrev[x])
counties_pop.drop(columns = 'GEO.display-label', inplace = True)
counties_pop.rename(columns = {'respop72016':'pop2016'}, inplace=True)

In [14]:
counties_pop.head()

,pop2016,County,State
0,55278,Autauga,AL
1,207509,Baldwin,AL
2,25774,Barbour,AL
3,22633,Bibb,AL
4,57562,Blount,AL


In [15]:
# Match BA to county: make table :  BAname-County-State-Population

In [16]:
ba_county.head()


,Data Year,Utility Number,Utility Name,State_x,County,State_y,BA_CODE,Megawatthours
0,2016.0,34,City of Abbeville - (SC),SC,Abbeville,NaN,NaN,NaN
1,2016.0,55,City of Aberdeen - (MS),MS,Monroe,AL,TVA,0
2,2016.0,55,City of Aberdeen - (MS),MS,Monroe,MS,TVA,201106
3,2016.0,59,City of Abbeville - (LA),LA,Vermilion,LA,MISO,132078
4,2016.0,84,A & N Electric Coop,MD,Somerset,MD,PJM,2565


In [17]:
ba_county_uniq = ba_county[['BA_CODE','County','State_x','State_y']].drop_duplicates(keep='first')
ba_county_uniq.drop(columns = ['State_y'], inplace=True)
ba_county_uniq.rename(columns = {'State_x':'State'}, inplace=True)
ba_county_uniq.dropna(axis = 0, how = 'any', inplace=True)
ba_county_uniq.drop_duplicates(keep = 'first', inplace=True)

In [18]:
ba_county_uniq.head()

,BA_CODE,County,State
1,TVA,Monroe,MS
3,MISO,Vermilion,LA
4,PJM,Somerset,MD
6,PJM,Accomack,VA
8,PJM,Northampton,VA


In [19]:
counties_pop.head()

,pop2016,County,State
0,55278,Autauga,AL
1,207509,Baldwin,AL
2,25774,Barbour,AL
3,22633,Bibb,AL
4,57562,Blount,AL


In [20]:
ba_counties_pop = pd.merge(ba_county_uniq, counties_pop, how = 'inner', on = ['County','State'])
ba_counties_pop['County_State'] = ba_counties_pop['County'] + ',' + ba_counties_pop['State']

In [21]:
ba_counties_pop.head()

,BA_CODE,County,State,pop2016,County_State
0,TVA,Monroe,MS,35908,"Monroe,MS"
1,PJM,Somerset,MD,25833,"Somerset,MD"
2,PJM,Accomack,VA,32818,"Accomack,VA"
3,PJM,Northampton,VA,12006,"Northampton,VA"
4,MISO,Adams,IL,66673,"Adams,IL"


In [22]:
ba_county_uniq['BA_CODE'].unique()

array(['TVA', 'MISO', 'PJM', 'NYIS', 'SOCO', 'SC', 'AZPS', 'FPC', 'CISO',
       'nan', 'DUK', 'SWPP', 'AMPL', 'AEC', 'LGEE', 'WALC', 'CPLE',
       'PACW', 'PSCO', 'AECI', 'ERCO', 'BPAT', 'NBSO', 'ISNE', 'WACM',
       'NWMT', 'PSEI', 'PACE', 'LDWP', 'PGE', 'SPA', 'SEC', 'PNM', 'CHPD',
       'AVA', 'CEA', 'NEVP', 'DOPD', 'EPE', 'EEI', 'FPL', 'JEA', 'FMPP',
       'SEPA', 'GVL', 'WAUW', 'HST', 'IPCO', 'IID', 'BANC', 'NSB', 'OVEC',
       'SCEG', 'GCPD', 'SRP', 'SCL', 'TPWR', 'TAL', 'TEC', 'TIDC', 'HECO',
       'TEPC'], dtype=object)

In [23]:
ba_counties_pop['BA_CODE'].unique()

array(['TVA', 'PJM', 'MISO', 'OVEC', 'NYIS', 'SOCO', 'SWPP', 'WALC',
       'WACM', 'PACW', 'SC', 'SCEG', 'DUK', 'AZPS', 'TEPC', 'AEC', 'FPC',
       'SEC', 'FPL', 'GVL', 'CISO', 'CPLE', 'AECI', 'LGEE', 'IID', 'PACE',
       'PNM', 'SRP', 'SPA', 'PSCO', 'ERCO', 'LDWP', 'EPE', 'BPAT', 'NBSO',
       'ISNE', 'FMPP', 'TEC', 'NWMT', 'WAUW', 'TPWR', 'GCPD', 'AVA',
       'PSEI', 'nan', 'NEVP', 'IPCO', 'PGE', 'CHPD', 'SCL', 'DOPD', 'JEA',
       'NSB', 'EEI', 'SEPA', 'TAL', 'BANC', 'TIDC', 'HECO'], dtype=object)

In [24]:
print(len(ba_county_uniq['BA_CODE'].unique()))
print(len(ba_counties_pop['BA_CODE'].unique()))

62
59


In [25]:
# Get WECC BAs

In [26]:
wecc_ba_counties_pop = ba_counties_pop.loc[ba_counties_pop['BA_CODE'].isin(ba_all)]

In [27]:
wecc_ba_counties_pop.shape

(1549, 5)

In [28]:
wecc_ba_counties_pop.loc[wecc_ba_counties_pop['State']=='NV'].sort_values('BA_CODE')


,BA_CODE,County,State,pop2016,County_State
3838,BPAT,Humboldt,NV,16873,"Humboldt,NV"
4284,BPAT,Elko,NV,52283,"Elko,NV"
4777,CISO,Washoe,NV,452429,"Washoe,NV"
4916,NEVP,Mineral,NV,4396,"Mineral,NV"
4915,NEVP,Lyon,NV,52854,"Lyon,NV"
4914,NEVP,Lander,NV,5765,"Lander,NV"
4913,NEVP,Esmeralda,NV,828,"Esmeralda,NV"
4912,NEVP,Douglas,NV,47947,"Douglas,NV"
4911,NEVP,Churchill,NV,24016,"Churchill,NV"
4910,NEVP,Carson City,NV,54283,"Carson City,NV"


In [29]:
wecc_ba_counties_pop.loc[wecc_ba_counties_pop['BA_CODE']=='IID']

,BA_CODE,County,State,pop2016,County_State
284,IID,Riverside,CA,2386522,"Riverside,CA"
3360,IID,Imperial,CA,180980,"Imperial,CA"
3366,IID,San Diego,CA,3317200,"San Diego,CA"


* Comparing with Balancing_Authority_2016.xlsx, BA='IID' is missing in NV list
* Comparing with Balancing_Authority_JAN17.pdf (which can arguably be lower resolution): only NEVP serves NV
* Document inconsistencies for later checking: which are real and which are due to the joins? (if any of BA, County, or State are missing, they drop off the list); How many counties are covered? For example, 'Fond du Lac, WI' is only present in county_pop file.

In [30]:
wecc_ba_counties_pop.loc[wecc_ba_counties_pop['State']=='UT']['BA_CODE'].unique()

array(['PACE', 'PACW', 'WACM', 'WALC', 'BPAT', 'NEVP'], dtype=object)

In [31]:
wecc_ba_counties_pop['BA_CODE'].unique()

array(['WALC', 'WACM', 'PACW', 'AZPS', 'TEPC', 'CISO', 'IID', 'PACE',
       'PNM', 'SRP', 'PSCO', 'LDWP', 'EPE', 'BPAT', 'NWMT', 'WAUW',
       'TPWR', 'GCPD', 'AVA', 'PSEI', 'NEVP', 'IPCO', 'PGE', 'CHPD',
       'SCL', 'DOPD', 'BANC', 'TIDC'], dtype=object)

In [32]:
len(wecc_ba_counties_pop['BA_CODE'].unique())

28

In [33]:
wecc_ba_counties_pop.loc[wecc_ba_counties_pop['BA_CODE']=='TEPC']

,BA_CODE,County,State,pop2016,County_State
69,TEPC,Pima,AZ,1012519,"Pima,AZ"
363,TEPC,Cochise,AZ,125355,"Cochise,AZ"
394,TEPC,Mohave,AZ,205385,"Mohave,AZ"
413,TEPC,Santa Cruz,AZ,46075,"Santa Cruz,AZ"


In [34]:
counties_pop.head()

,pop2016,County,State
0,55278,Autauga,AL
1,207509,Baldwin,AL
2,25774,Barbour,AL
3,22633,Bibb,AL
4,57562,Blount,AL


In [35]:
ba_counties_pop.loc[ba_counties_pop['BA_CODE']== 'EPE']['County'].tolist()

['Uvalde',
 'Eddy',
 'Otero',
 'Luna',
 'Sierra',
 'Culberson',
 'El Paso',
 'Hudspeth',
 'Dimmit',
 'Edwards',
 'Kinney',
 'Webb',
 'Zavala',
 'Brewster',
 'Crockett',
 'Jeff Davis',
 'Maverick',
 'Pecos',
 'Presidio',
 'Reeves',
 'Terrell',
 'Val Verde']

In [36]:
# Load Zone mapping: use county-pop file; except CA and ElPaso
wecc_states_list = ['WA','OR','CA','NV','AZ','UT','NM','CO','WY','MT']

zone_county = {}
for r in wecc_states_list:
    zone_county[r] = counties_pop.loc[counties_pop['State'] == r]['County'].tolist()

zone_county['Northern_California'] = ['Butte','Colusa','Del Norte','El Dorado','Glenn','Humboldt',
                       'Lake','Lassen','Mendocino','Modoc','Nevada','Placer','Plumas',
                       'Sacramento','Shasta','Sierra','Siskiyou','Sutter','Tehama',
                       'Trinity','Yolo','Yuba']
zone_county['Bay_Area'] = ['Alameda','Contra Costa','Marin','Napa','San Francisco','San Mateo',
            'Santa Clara','Santa Cruz','Solano','Sonoma']
zone_county['Central_California'] = ['Alpine','Amador','Calaveras','Fresno','Inyo','Kings','Madera',
                      'Mariposa','Merced','Mono','Monterey','San Benito','San Joaquin',
                      'Stanislaus','Tulare','Tuolumne']
zone_county['Southeast_California'] = ['Imperial','Orange','Riverside','San Bernardino','San Diego']
zone_county['Southwest_California'] = ['Kern','Los Angeles','San Luis Obispo','Santa Barbara','Ventura']

#El Paso is defined by BA= EPE, so counties served by EPE
zone_county['El_Paso'] = ba_counties_pop.loc[ba_counties_pop['BA_CODE']== 'EPE']['County'].tolist()

* WA = ['AVA', 'BPAT','CHPD','DOPD','GCPD','PSEI','SCL','TPWR','AVA', 'BPAT']
* OR = ['BPAT','GRID','IPCO','PACW','PGE']
* CAnorth = ['BANC','CISO','PACW','BPAT']
* CABayArea = ['CISO']
* CAcentral = ['CISO', 'TIDC']
* CAsw = ['CISO', 'LDWP']
* CAse = ['CISO', 'IID']
* NV = ['NEVP']
* AZ = ['AZPS','DEAA','GRIF','GRMA','HGMA','PNM','SRP','TEPC','WALC']
* UT = ['PACE']
* NM = ['EPE','PNM']
* CO = ['PSCO','WACM']
* WY = ['PACE','WACM']
* ID = ['AVA','BPAT','IPCO','PACE']
* MT = ['BPAT','GWA','NWMT','WAUW','WWA'] 
* ElPaso = ['EPE']

In [37]:
wecc_ba_counties_pop.head()

,BA_CODE,County,State,pop2016,County_State
37,WALC,Stevens,MN,9747,"Stevens,MN"
38,WACM,Stevens,MN,9747,"Stevens,MN"
39,PACW,Stevens,MN,9747,"Stevens,MN"
41,WALC,Swift,MN,9444,"Swift,MN"
42,WACM,Swift,MN,9444,"Swift,MN"


In [38]:
def assign_demand_by_county(ba_county_pop, zone_county_dict, ba_demand):
    '''
    ba_county_pop = dataframe containing BA-county-population mapping
    zone_county_dict = dict containing zone-counties mapping
    ba_demand = dataframe of counts of all BAs for 2016
    '''
    
    
    #for each load zone:
    

    
    
    
    
    
    
    

In [42]:
BA_list = ba_counties_pop['BA_CODE'].unique()
ba_county_dict = {}

for ba in BA_list:
    ba_county_dict[ba] = ba_counties_pop.loc[ba_counties_pop['BA_CODE']==ba]['County_State']

In [44]:
ba_county_dict['EPE']

612         Uvalde,TX
1507          Eddy,NM
1511         Otero,NM
1948          Luna,NM
2472        Sierra,NM
2474     Culberson,TX
2476       El Paso,TX
2478      Hudspeth,TX
3952        Dimmit,TX
3955       Edwards,TX
3958        Kinney,TX
3960          Webb,TX
3963        Zavala,TX
4843      Brewster,TX
4845      Crockett,TX
4847    Jeff Davis,TX
4849      Maverick,TX
4851         Pecos,TX
4853      Presidio,TX
4855        Reeves,TX
4857       Terrell,TX
4859     Val Verde,TX
Name: County_State, dtype: object

In [ ]:
ba_counties_pop.head()

In [ ]:
ba_counties_pop['County'] + ',' + ba_counties_pop['State']